In [11]:
import musx
import rtmidi
from custom_utils import *

In [2]:
midiout = rtmidi.MidiOut()
outports = midiout.get_ports()
print("available ports:", outports)

available ports: ['Microsoft GS Wavetable Synth 0', 'musx2reaper 1']


In [3]:
midiout.open_port(outports.index('musx2reaper 1'))

In [4]:
def pickrand(index, total):
    if (index / total) < 0.33:
        return musx.lowran()
    elif (index / total) < 0.66:
        return musx.midran()
    else:
        return musx.highran()

def random_ascend(queue, num, root, rhy, dur, amp, chan):
    scale = musx.scale(root, 17, 2, 2, 1, 2, 2, 2, 1) # 2 octave major scale

    rhythms = musx.choose([rhy, [0, rhy/2], [0, rhy/4, rhy/2, 3 * (rhy/4)], [0, 3 * (rhy/4)], [0, 0], [0, 0, rhy/2, rhy/2], [0, 0, 0]]) # repeated ones are a hack for triads
    for i in range(num):
        r = next(rhythms)
        if musx.isnum(r):
            n = int(musx.rescale(pickrand(i, num), 0, 1, 0, len(scale)))
            k = scale[n]
            m = musx.MidiNote(time=queue.now, dur=rhy, key=k, amp=amp, chan=chan)
            queue.out.addevent(m)
        else:
            for time in r:
                n = int(musx.rescale(pickrand(i, num), 0, 1, 0, len(scale)))
                k = scale[n]
                m = musx.MidiNote(time=queue.now + time, dur=rhy, key=k, amp=amp, chan=chan)
                queue.out.addevent(m)
        yield rhy

In [5]:
def cc_init(queue):
    queue.out.addevent(musx.MidiEvent.control_change(0, 32, 0)) # Reset Supermassive mix to 0%
    # queue.out.addevent(musx.MidiEvent.control_change(0, 32, 64))
    queue.out.addevent(musx.MidiEvent.control_change(0, 33, 64)) # Reset bottle-high panning to center
    queue.out.addevent(musx.MidiEvent.control_change(0, 34, 64)) # Reset bottle-high panning to center
    yield 0

In [6]:
def cc_distribution(queue, chan, ctrl, *, num, rate, distribution=musx.uniran, low=0, high=1):
    for i in range(num):
        original = max(low, min(distribution(), high))
        val = int(musx.rescale(original, low, high, 0, 127))
        queue.out.addevent(musx.MidiEvent.control_change(chan, ctrl, val, time=queue.now + (i * rate)))
        yield rate

In [7]:
def euclidean_rhythm(queue, length, keynum, events, steps, rotation, cycletime, amp, chan):
    sequence = bjorklund(events, steps)
    start_time = queue.now
    notelen = cycletime / steps
    i = rotation
    while queue.now - start_time < length:
        if sequence[i] == 1:
            queue.out.addevent(musx.MidiNote(time=queue.now, dur=notelen, key=keynum, amp=amp, chan=chan))
        i += 1
        i %= steps
        yield notelen

In [9]:
seq = musx.MidiSeq()
queue = musx.Scheduler(out=seq)
queue.compose([[0, cc_init(queue)],
               [0, random_ascend(queue, 120, 72, 0.5, 1, 0.25, 0)],
               [0, cc_distribution(queue, 0, 33, num=120, rate=0.25)],
               [0, random_ascend(queue, 60, 60, 1, 1, 0.25, 1)],
               [0, cc_distribution(queue, 0, 34, num=60, rate=0.5)],
            #    [0, random_ascend(queue, 120, 48, 0.5, 1, 0.25, 2)],
               [30, cc_linear(queue, 0, 32, length=30, start=0, end=0.25)]
               ])
seq.play(midiout, False)
# musx.MidiFile("random_ascend.mid", seq).write()

Done!


In [8]:
seq = musx.MidiSeq()
queue = musx.Scheduler(out=seq)
queue.compose([[0, cc_init(queue)],
               [0, euclidean_rhythm(queue, 16, 60, 3, 16, 0, 4, 0.25, 2)],
               [0, euclidean_rhythm(queue, 16, 64, 7, 16, 0, 4, 0.25, 2)],
               [0, euclidean_rhythm(queue, 16, 67, 3, 16, 9, 4, 0.25, 2)],
               [0, euclidean_rhythm(queue, 16, 70, 7, 16, 7, 4, 0.25, 2)]])
seq.play(midiout, False)

Done!


In [13]:
midiout = midi_restart(midiout, 'musx2reaper 1')

In [9]:
# midiout.send_message(musx.control_change(0, 32, 64)) # used for MIDI learning the Supermassive parameter
# midiout.send_message(musx.control_change(0, 33, 64)) # used for MIDI learning bottle-high pan parameter
midiout.send_message(musx.control_change(0, 34, 64)) # used for MIDI learning bottle-high pan parameter

In [29]:
musx.gauss()

-1.1847760540811445